In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('breast_cancer.csv')

In [8]:
df.head()

In [9]:
df.shape

In [10]:
df.isnull().sum()

In [11]:
df = df.drop(['id', 'Unnamed: 32'], axis=1)

In [ ]:
df.head()

In [ ]:
df.select_dtypes(exclude='number')

In [ ]:
def convert(data):
    if data == 'M':
        return 1
    else:
        return 0
df['diagnosis'] = df['diagnosis'].apply(convert)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(6, 4), dpi=150)
sns.countplot(x=df['diagnosis'])

In [ ]:
df.corr()['diagnosis'].sort_values()

In [ ]:
plt.figure(figsize=(6, 4), dpi=150)
sns.scatterplot(x=df['concavity_worst'], y=df['perimeter_worst'], hue=df['diagnosis'])

In [ ]:
imp_features = ['perimeter_mean', 'radius_worst', 'concave points_mean', 'perimeter_worst', 'concave points_worst', 'diagnosis']

In [ ]:
sns.pairplot(df[imp_features], hue='diagnosis')

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [ ]:
model1 = DecisionTreeClassifier()

In [ ]:
param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth' : [6, 7, 8]}

grid = GridSearchCV(model1, param_grid)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
model2 = DecisionTreeClassifier(max_depth=7, max_features='auto')

In [ ]:
model2.fit(X_train, y_train)

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
model3 = AdaBoostClassifier(n_estimators=9)

In [ ]:
model3.fit(X_train, y_train)

In [ ]:
model3.feature_importances_

In [ ]:
y_pred = model3.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
df_imp_features = pd.DataFrame(index=X.columns, data=model3.feature_importances_, columns=['Importance'])

In [ ]:
df_imp_features['Importance'] = round(df_imp_features['Importance'], 2)

In [ ]:
dff = df_imp_features[df_imp_features['Importance'] != 0].sort_values('Importance')

In [ ]:
dff

In [ ]:
plt.figure(figsize=(6, 6), dpi=150)
sns.barplot(x=dff.index, y=dff['Importance'])
plt.xticks(rotation=90);

In [ ]:
df[['texture_mean', 'compactness_se', 'texture_worst', 'area_worst',
       'smoothness_worst', 'concavity_worst', 'concave points_worst',
       'area_se']].describe()

In [ ]:
# Loaded model 
m = {}
for i in X.columns:
    m[i] = 0

texture_mean = float(input("Enter the texture mean range(9.71-39.28): "))
m['texture_mean'] = texture_mean

compactness_se = float(input("Enter the compactness_se range(0.002-0.135): "))
m['compactness_se'] = compactness_se

texture_worst = float(input("Enter the texture_worst range(12.02-49.54): "))
m['texture_worst'] = texture_worst

area_worst = float(input("Enter the area_worst range(185.2-4254.0): "))
m['area_worst'] = area_worst

smoothness_worst = float(input("Enter the smoothness_worst range(0.07-0.22): "))
m['smoothness_worst'] =smoothness_worst

concavity_worst = float(input("Enter the concavity_worst range(0.00-1.25): "))
m['concavity_worst'] = concavity_worst

concave_points_worst = float(input("Enter the concave points_worst range(0.00-0.29): "))
m['concave points_worst'] = concave_points_worst

area_se = float(input("Enter the area_se range(6.80-542.20): "))
m['area_se'] = area_se


predictors = [list(m.values())]
outcome = model3.predict(predictors)[0]
prob = model3.predict_proba(predictors)[0]

if outcome == 0:
    print("You don't have Breast-Cancer: ")
    print(f"Probablity of the outcome is {round(prob[0]*100, 2)}%. ")
else:
    print("You have Breast-Cancer: ")
    print(f"Probablity of the outcome is {round(prob[1]*100, 2)}%. ")
    
if outcome == 1 and prob[1] > 0.80:
    print(f"Consult Doctor Immediately. ")




In [ ]:
from joblib import dump, load

In [ ]:
dump(model3, "Breast-Cancer.joblib")